In [1]:
import io
import re
import unicodedata as ud
import pandas as pd

In [2]:
easy_sentence = io.open("./data/easy_sentence.txt", encoding="utf-8").readlines()
hard_sentence = io.open("./data/hard_sentence.txt", encoding="utf-8").readlines()

In [3]:
data_sentence = easy_sentence
data_sentence.extend(hard_sentence)
data_sentence[:5]

['thành phố washington có một kiến trúc rất đa dạng\n',
 'tuy nhiên vì gặp nhiều khó khăn trong cuộc sống ông dần trở nên khó tính\n',
 'khí hậu hồng kông thuộc kiểu cận nhiệt đới và chịu ảnh hưởng của gió mùa\n',
 'khoảng hơn 70 bề mặt trái đất được bao phủ bởi các đại dương nước mặn phần còn lại là các lục địa và các đảo\n',
 'đà lạt là thành phố trực thuộc tỉnh lâm đồng nằm trên cao nguyên lâm viên thuộc vùng tây nguyên việt nam\n']

In [4]:
def syllablize(sentence):
    word = '\w+'
    non_word = '[^\w\s]'
    digits = '\d+([\.,_]\d+)+'
    
    patterns = []
    patterns.extend([word, non_word, digits])
    patterns = f"({'|'.join(patterns)})"
    
    sentence = ud.normalize('NFC', sentence)
    tokens = re.findall(patterns, sentence, re.UNICODE)
    return [token[0] for token in tokens]

In [5]:
def load_n_grams(path):
    with open(path, encoding='utf8') as f:
        words = f.read().splitlines() 
    return words

# Longest Matching

In [6]:
def longest_matching(sentence, bi_grams, tri_grams):
    syllables = syllablize(sentence)
    syl_len = len(syllables)

    curr_id = 0
    word_list = []
    done = False
    
    while (curr_id < syl_len) and (not done):
        curr_word = syllables[curr_id]
        if curr_id >= syl_len - 1:
            word_list.append(curr_word)
            done = True
        else:
            next_word = syllables[curr_id + 1]
            pair_word = ' '.join([curr_word.lower(), next_word.lower()])
            if curr_id >= (syl_len - 2):
                if pair_word in bi_grams:
                    word_list.append('_'.join([curr_word, next_word]))
                    curr_id += 2
                else:
                    word_list.append(curr_word)
                    curr_id += 1
            else:
                next_next_word = syllables[curr_id + 2]
                triple_word = ' '.join([pair_word, next_next_word.lower()])
                if triple_word in tri_grams:
                    word_list.append('_'.join([curr_word, next_word, next_next_word]))
                    curr_id += 3
                elif pair_word in bi_grams:
                    word_list.append('_'.join([curr_word, next_word]))
                    curr_id += 2
                else:
                    word_list.append(curr_word)
                    curr_id += 1
    return word_list

In [7]:
bi_grams = load_n_grams('./Vocab/vocab_bi_gram.txt')
tri_grams = load_n_grams('./Vocab/vocab_tri_gram.txt')

In [8]:
with open('Token/longest_matching_tokens.txt', 'w', encoding='utf-8') as f:
    longest_matching_sentences = []
    for sentence in data_sentence:
        word_list = longest_matching(sentence, bi_grams, tri_grams)
        longest_matching_sentences.append(' '.join(word_list))
        f.write(' '.join(word_list) + '\n')
longest_matching_sentences[0:3]

['thành_phố washington có một kiến_trúc rất đa_dạng',
 'tuy_nhiên vì gặp nhiều khó_khăn trong cuộc_sống ông dần trở_nên khó_tính',
 'khí_hậu hồng_kông thuộc kiểu cận nhiệt_đới và chịu ảnh_hưởng của gió mùa']

In [9]:
count_longest_matching_compounds = 0
for sentence in longest_matching_sentences:
    for word in sentence.split():
        if '_' in word: count_longest_matching_compounds += 1
print('Số lượng từ ghép khi tách từ bằng thuật toán Longest Matching:', count_longest_matching_compounds)

Số lượng từ ghép khi tách từ bằng thuật toán Longest Matching: 224


# VnCoreNLP

In [10]:
import py_vncorenlp
model = py_vncorenlp.VnCoreNLP(annotators=["wseg", "pos"], save_dir='./')

In [11]:
with open('Token/vncore_tokens.txt', 'w', encoding='utf-8') as f:
    vncore_sentences = []
    for sentence in data_sentence:
        words = model.word_segment(sentence)[0]
        vncore_sentences.append(words)
        f.write(words + '\n')
vncore_sentences[0:3]

['thành_phố washington có một kiến_trúc rất đa_dạng',
 'tuy_nhiên vì gặp nhiều khó_khăn trong cuộc_sống ông dần trở_nên khó_tính',
 'khí_hậu hồng_kông thuộc kiểu cận_nhiệt_đới và chịu ảnh_hưởng của gió_mùa']

In [12]:
count_vncore_compounds = 0
for sentence in vncore_sentences:
    for word in sentence.split():
        if '_' in word: count_vncore_compounds += 1
print('Số lượng từ ghép khi tách từ bằng thư viện VnCoreNLP:', count_vncore_compounds)

Số lượng từ ghép khi tách từ bằng thư viện VnCoreNLP: 231


# Gold Data

In [13]:
gold_sentence = io.open("./Data/gold_sentence.txt", encoding="utf-8").readlines()

In [14]:
manual_tokenize_sentences = []
for sentence in gold_sentence:
    if sentence != '\n': 
        manual_tokenize_sentences.append(sentence.strip())
manual_tokenize_sentences[0:3]

['thành_phố washington có một kiến_trúc rất đa_dạng',
 'tuy_nhiên vì gặp nhiều khó_khăn trong cuộc_sống ông dần trở_nên khó_tính',
 'khí_hậu hồng_kông thuộc kiểu cận_nhiệt_đới và chịu ảnh_hưởng của gió_mùa']

In [15]:
count_manual_tokenize_compounds = 0
for sentence in manual_tokenize_sentences:
    for word in sentence.split():
        if '_' in word: count_manual_tokenize_compounds += 1
print('Số lượng từ ghép khi tách từ thủ công:', count_manual_tokenize_compounds)

Số lượng từ ghép khi tách từ thủ công: 255


# Đánh giá

In [16]:
def count_correct_words(pred, source, n_grams=3):
    pred_words = pred.split()
    source_words = source.split()
    
    total_true, tp = 0, 0
    total_errors, fp = 0, 0
    
    idx = 0
    while idx < len(pred_words):
        if pred_words[idx] not in source_words[idx:(idx + n_grams)]: 
            if '_' in pred_words[idx]: fp += 1
            del pred_words[idx]
            total_errors += 1
        else: idx += 1
    
    idx = 0
    while idx < len(source_words):
        if source_words[idx] not in pred_words[idx:(idx + n_grams)]: 
            del source_words[idx]
        else: idx += 1
    
    if len(pred_words) < len(source_words): words = pred_words
    else: words = source_words
    
    for idx in range (len(words)):
        if pred_words[idx] == source_words[idx]:
            if '_' in pred_words[idx]: tp += 1 
            total_true += 1
                    
    return total_true, total_errors, tp, fp

In [17]:
def tokenize_evaluation(pred, source, n_grams=3):
    total_true = 0
    total_errors = 0
    total_words = 0
    
    pred_tp = 0
    pred_fp = 0
    
    for pred_sentence, source_sentence in zip(pred, source):
        total_words += len(source_sentence.split())
        if pred_sentence != source_sentence:
            true, error, tp, fp = count_correct_words(pred_sentence, source_sentence, n_grams)
            total_true += true 
            total_errors += error
            pred_tp += tp
            pred_fp += fp
        else:
            for word in source_sentence.split():
                if '_' in word:
                    pred_tp += 1
                total_true += 1
                    
    accuracy = total_true / total_words
    precision = pred_tp / (pred_tp + pred_fp)
    recall = pred_tp / count_manual_tokenize_compounds
    f1 = 2 * precision * recall / (precision + recall)
    return {
        'Accuracy': accuracy, 
        'Precision': precision,
        'Recall': recall,
        'F1': f1,
        'True Positive': pred_tp, 
        'False Positive': pred_fp,
        'Total True': total_true, 
        'Total Errors': total_errors
    }

In [18]:
longest_matching_evaluation = tokenize_evaluation(longest_matching_sentences, manual_tokenize_sentences)
vncore_evaluation = tokenize_evaluation(vncore_sentences, manual_tokenize_sentences)
pd.DataFrame(
    [longest_matching_evaluation, vncore_evaluation], 
    index = ['Longest Matching', 'VnCoreNLP']
).astype(object).T

,Longest Matching,VnCoreNLP
Accuracy,0.90625,0.915761
Precision,0.950893,0.943723
Recall,0.835294,0.854902
F1,0.889353,0.897119
True Positive,213,218
False Positive,11,13
Total True,667,674
Total Errors,97,79
